In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import HashingVectorizer

In [9]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
data = import_data("./data/parking_citations.corrupted.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3209: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Memory usage of dataframe is 1264.91 MB
Memory usage after optimization is: 840.01 MB
Decreased by 33.6%


In [10]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1) 
       
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [11]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [12]:
missing_values_table(data)

,Missing Values,% of Total Values
VIN,8709705,99.8
Marked Time,8435415,96.7
Meter Id,6456512,74.0
Make,4368470,50.1
Plate Expiry Date,794827,9.1
Route,65354,0.7
Body Style,8890,0.1
Fine amount,6507,0.1
Color,4115,0.0
Issue time,2583,0.0


In [13]:
make = data['Make']
data = data.drop(['Make'], axis=1)

In [14]:
missing_values_table(data)

,Missing Values,% of Total Values
VIN,8709705,99.8
Marked Time,8435415,96.7
Meter Id,6456512,74.0
Plate Expiry Date,794827,9.1
Route,65354,0.7
Body Style,8890,0.1
Fine amount,6507,0.1
Color,4115,0.0
Issue time,2583,0.0
Violation Description,872,0.0


In [15]:
data[data["VIN"].isna()==False]

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
1235876,4241125932,2014-10-31T00:00:00,805.0,NaN,NaN,CA,NaN,1ZVFT80N465228314,PA,BK,7045 LINDLEY AVE,00374,53.0,165,NO PARK/STREET CLEAN,73.0,6402218.0,1894969.750
1235877,4241125943,2014-10-31T00:00:00,807.0,NaN,NaN,CA,201509.0,0199,PA,SL,6941 LINDLEY AVE,00374,53.0,165,NO PARK/STREET CLEAN,73.0,6402214.5,1894293.000
1235878,4241125954,2014-10-31T00:00:00,809.0,NaN,NaN,CA,201502.0,3491,PA,BK,6621 LINDLEY AVE,00374,53.0,165,NO PARK/STREET CLEAN,73.0,6402205.5,1892169.750
1235879,4241125965,2014-10-31T00:00:00,814.0,NaN,NaN,CA,NaN,1G1ZS58F47F170546,PA,SL,6430 RESEDA BLVD,00374,53.0,165,NO PARK/STREET CLEAN,73.0,6399558.0,1890949.875
1235880,4241125976,2014-10-31T00:00:00,815.0,NaN,NaN,CA,201412.0,9817,PA,GN,6430 RESEDA BLVD,00374,53.0,165,NO PARK/STREET CLEAN,73.0,6399558.0,1890949.875
1235881,4241125980,2014-10-31T00:00:00,819.0,NaN,NaN,CA,201508.0,0739,PA,SL,18325 HART ST,00374,53.0,165,NO PARK/STREET CLEAN,73.0,6400761.5,1894682.000
1235882,4241125991,2014-10-31T00:00:00,820.0,NaN,NaN,CA,201504.0,3204,PU,BL,18325 HART ST,00374,53.0,165,NO PARK/STREET CLEAN,73.0,6400761.5,1894682.000
1235883,4241126002,2014-10-31T00:00:00,820.0,NaN,NaN,CA,201510.0,NOT VISIBLE,PA,GY,18325 HART ST,00374,53.0,165,NO PARK/STREET CLEAN,73.0,6400761.5,1894682.000
1235884,4242202285,2014-11-01T00:00:00,613.0,NaN,NaN,AZ,NaN,NOT VISIBLE,TR,WT,APPX 1073 12TH ST W,00500,56.0,155,STANDNG IN ALLEY,68.0,6470260.5,1725482.875
1235885,4242202296,2014-11-01T00:00:00,618.0,NaN,NaN,AZ,201410.0,0003,PU,WT,1073 WEST 12TH STREET,00500,56.0,155,STANDNG IN ALLEY,68.0,6470260.5,1725482.875


In [16]:
data["VIN"].fillna("NOT VISIBLE", inplace=True)

In [17]:
#need to redo this with k means
marked_time = data["Marked Time"]
data["Marked Time"].fillna(data["Marked Time"].mean, inplace=True)

In [18]:
#needs to be k means clustered
meter_id = data["Meter Id"]
data["Meter Id"].fillna("!!", inplace=True)

In [19]:
plate_expiry_date = data["Plate Expiry Date"]
data["Plate Expiry Date"].fillna(data["Plate Expiry Date"].mean, inplace=True)

In [20]:
#kmeans 
route = data["Route"]
data["Route"].fillna("00600", inplace=True)

In [21]:
body_style = data["Body Style"]
data['Body Style'].fillna("PA", inplace=True)

In [22]:
#kmeans
fine_amount = data["Fine amount"]
data["Fine amount"].fillna(data["Fine amount"].mean(), inplace=True)

In [23]:
#kmeans
color = data["Color"]
data["Color"].fillna("BK", inplace=True)

In [24]:
issue_time = data["Issue time"]
data["Issue time"].fillna(data["Issue time"].mean(), inplace=True)

In [25]:
data["Violation Description"].cat.add_categories("NO DESCRIPTION", inplace=True)
data["Violation Description"].fillna("NO DESCRIPTION", inplace=True)

In [26]:
location = data["Location"]
data["Location"].fillna("1301 ELECTRIC AVE", inplace=True)

In [27]:
RP_state_plate = data["RP State Plate"]
data["RP State Plate"].fillna("CA", inplace=True)

In [28]:
agency = data["Agency"]
data["Agency"].fillna(data["Agency"].mean(), inplace=True)

In [29]:
#kmeans
issue_date = data["Issue Date"]
data["Issue Date"].fillna("2016-01-19T00:00:00", inplace=True)

In [30]:
issue_date = data["Issue Date"]
data["Issue Date"].fillna("2016-01-19T00:00:00", inplace=True)

In [31]:
latitude = data["Latitude"]
longitude = data["Longitude"]
data["Latitude"].fillna("99999.0", inplace=True)
data["Longitude"].fillna("99999.0", inplace=True)

In [26]:
make
#test = make[make['Make'].isna()]
#train = make[make['Make'].isna()==False]

0           NaN
1           NaN
2           NaN
3           NaN
4          CHEV
5           NaN
6          MAZD
7           NaN
8           NaN
9           NaN
10         FORD
11         CHRY
12          NaN
13          NaN
14         TOYO
15          NaN
16          NaN
17          NaN
18         CHEV
19         HOND
20          NaN
21         NISS
22         HOND
23          NaN
24          NaN
25         FORD
26          NaN
27          NaN
28          NaN
29         TESL
           ... 
8725984    TOYT
8725985     NaN
8725986     NaN
8725987     NaN
8725988     NaN
8725989    FORD
8725990     NaN
8725991     NaN
8725992    AUDI
8725993     NaN
8725994     NaN
8725995     NaN
8725996    VOLK
8725997     NaN
8725998    HOND
8725999    HOND
8726000    HOND
8726001     NaN
8726002     NaN
8726003    MERZ
8726004     NaN
8726005    PORS
8726006    FORD
8726007     NaN
8726008    PORS
8726009     NaN
8726010    HYUN
8726011    NISS
8726012     NaN
8726013     BMW
Name: Make, Length: 8726

In [27]:
data.dtypes

Ticket number            category
Issue Date               category
Issue time                float16
Meter Id                 category
Marked Time                object
RP State Plate           category
Plate Expiry Date          object
VIN                      category
Body Style               category
Color                    category
Location                 category
Route                    category
Agency                    float16
Violation code           category
Violation Description    category
Fine amount               float16
Latitude                   object
Longitude                  object
dtype: object

In [32]:
hashing = HashingVectorizer(lowercase=True, stop_words='english')

In [33]:
data["Violation Description"] = hashing.fit_transform(data["Violation Description"])

In [58]:
data['Location'] = hashing.fit_transform(data['Location'])

In [35]:
le = LabelEncoder()
data["Meter Id"] = le.fit_transform(data["Meter Id"]) 
#MultiColumnLabelEncoder(columns=["Meter Id", "RP State Plate", "VIN", "Body Style", "Color", "Route", "Violation code"]).fit_transform(data)

In [36]:
data["RP State Plate"] = le.fit_transform(data["RP State Plate"])

In [37]:
data["VIN"] = le.fit_transform(data["VIN"])

In [38]:
data["Body Style"] = le.fit_transform(data["Body Style"])

In [39]:
data["Color"] = le.fit_transform(data["Color"])

In [40]:
data["Route"] = le.fit_transform(data["Route"])

In [41]:
data["Violation code"] = le.fit_transform(data["Violation code"])

In [ ]:
clusters = 3
while clusters < 20:
    k = KMeans(n_clusters = clusters, n_jobs = -1)
    preds = k.fit_predict(data.values)
    sil = silhouette_score(data, preds)
    print(clusters, sil)
    clusters +=1